In [ ]:
!pip install opencv-python easyocr layoutparser pdf2image

In [ ]:
import cv2
import numpy as np
import easyocr
import re
from google.colab import files
from PIL import Image

# function to preprocess and extract answers from the image using OCR
def preprocess_and_extract_answers(image_path):
    #load the iumage
    image = Image.open(image_path)

    #convert to graysscale for better OCR results
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)

    #initialize EasyOCR Reader
    reader = easyocr.Reader(['en'])

    #perform OCR to exxrtract text from the image
    result = reader.readtext(gray_image)

    #extract teext from the OCR results
    extracted_text = []
    for line in result:
        extracted_text.append(line[1])

    return extracted_text

#function to paarse extracted text (answers)
def parse_answers(extracted_text):
    answers = []
    #Ue regex to identify answer patterns
    for line in extracted_text:
        match = re.match(r'(\d+)\.\s*([A-D])', line)  # Match answer patterns (1. A)
        if match:
            answers.append(match.group(2))  #append the recognized answer (A, B, C, D)
    return answers

#fununction to grade answers by comparing with the answer key
def grade_answers(extracted_answers, answer_key):
    correct_count = 0
    for i in range(min(len(answer_key), len(extracted_answers))):
        if extracted_answers[i] == answer_key[i]:
            correct_count += 1

    score = (correct_count / len(answer_key)) * 100
    return correct_count, len(answer_key), score

#function to process the image and output the grading result
def process_image(image_path, answer_key):
    #preprocess the image and extract answers
    extracted_text = preprocess_and_extract_answers(image_path)

    #parse extracted text into answers
    extracted_answers = parse_answers(extracted_text)

    #grade the answers
    correct_count, total_answers, score = grade_answers(extracted_answers, answer_key)

    #results
    return extracted_answers, correct_count, total_answers, score

#upload the image using coolab file upload feature
def upload_image():
    uploaded = files.upload()  # Upload image file
    for filename in uploaded.keys():
        return filename

#main function
def main():
    # Upload the image file
    print("Please upload an image of the worksheet to grade.")
    image_path = upload_image()

    #enter answer key here
    answer_key = ["A", "C", "D", "B", "A", "C", "D"]

    print(f"Processing image: {image_path}")

    #process the uploaded image and grade it
    extracted_answers, correct_count, total_answers, score = process_image(image_path, answer_key)

    #dislpay results
    print("\nResults:")
    print(f"Extracted Answers: {', '.join(extracted_answers)}")
    print(f"Correct Answers: {correct_count}/{total_answers}, Score: {score:.2f}%")

#run prgorams
if __name__ == "__main__":
    main()


Please upload an image of the worksheet to grade.


Saving Screenshot 2024-11-14 at 8.25.56 PM.png to Screenshot 2024-11-14 at 8.25.56 PM.png
Processing image: Screenshot 2024-11-14 at 8.25.56 PM.png
